# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

In [1]:
print("Julia Arwari")

Julia Arwari


## 0. Import relevant packages

In [3]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [90]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.

https://github.com/jarwari/BIOS-584-test

## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [6]:
# Write your own code
ptsd_dir = '{}/data/PTSD dataset.xlsx'.format(os.getcwd())
ptsd_df = pd.read_excel(ptsd_dir, sheet_name=1)

### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [13]:
# Write your own code
pre_score = 'pcl5week_score.completion'
post_score = 'pcl5month_score.3_month_follow_up'

def summarize(series):
    return {
        'mean': np.mean(series),
        'std': np.std(series, ddof=1),
        'median': np.median(series),
        'Q1': np.percentile(series, 25),
        'Q3': np.percentile(series, 75)
    }

summary_pre = summarize(ptsd_df[pre_score].dropna())
summary_post = summarize(ptsd_df[post_score].dropna())

print("\nSummary Statistics:")
print("Pre-Intervention (Completion):", summary_pre)
print("Post-Intervention (3-Month Follow-up):", summary_post)

ptsd_clean = ptsd_df[[pre_score, post_score]].dropna()
t_stat, p_val = stats.ttest_rel(ptsd_clean[pre_score], ptsd_clean[post_score])

print("\nPaired t-test results:")
print(f"t-statistic = {t_stat:.3f}, p-value = {p_val:.4f}")


Summary Statistics:
Pre-Intervention (Completion): {'mean': np.float64(29.43859649122807), 'std': np.float64(19.373839884545028), 'median': np.float64(25.0), 'Q1': np.float64(15.0), 'Q3': np.float64(42.0)}
Post-Intervention (3-Month Follow-up): {'mean': np.float64(31.719827586206897), 'std': np.float64(18.812907606503504), 'median': np.float64(29.5), 'Q1': np.float64(16.0), 'Q3': np.float64(44.25)}

Paired t-test results:
t-statistic = -4.709, p-value = 0.0000


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [16]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']
ptsd_sub_df = ptsd_df[relevant_col_names].dropna()

In [19]:
# Write your own code
formula = (
    "caps_intake ~ age_iop + C(gender_code) + C(sexualorient_code) + C(race_code) + "
    "C(ethnicity_code) + C(education_code) + C(employment_code) + C(rank_code) + "
    "C(branch_code) + C(mdd_code) + ctq_total_score + sexual_trauma + C(sud_code)"
)
model = smf.ols(formula=formula, data=ptsd_sub_df).fit()
print("\nR-squared:", round(model.rsquared, 3))
print("Adjusted R-squared:", round(model.rsquared_adj, 3))
print("\nParameter Estimates:\n")
print(model.params)
print("\n95% Confidence Intervals:\n")
print(model.conf_int())
print("\nP-values:\n")
print(model.pvalues)


R-squared: 0.114
Adjusted R-squared: 0.051

Parameter Estimates:

Intercept                    39.262913
C(gender_code)[T.2]          -1.558164
C(sexualorient_code)[T.2]     1.824729
C(sexualorient_code)[T.3]     1.801214
C(race_code)[T.2]             0.532547
C(race_code)[T.3]            -3.289156
C(race_code)[T.4]             3.236026
C(race_code)[T.5]            -2.890123
C(race_code)[T.6]            -3.546662
C(ethnicity_code)[T.2]       -2.883958
C(ethnicity_code)[T.3]       -2.516272
C(education_code)[T.2]       -1.359642
C(education_code)[T.3]        4.298667
C(employment_code)[T.2]      -0.273740
C(employment_code)[T.3]      -0.436353
C(employment_code)[T.4]       2.740779
C(employment_code)[T.5]      -1.696072
C(employment_code)[T.6]      -2.015037
C(rank_code)[T.2]            -3.055898
C(rank_code)[T.3]            -2.490140
C(branch_code)[T.2]           0.179519
C(branch_code)[T.3]           0.975998
C(branch_code)[T.4]          -2.210443
C(branch_code)[T.5]          -0.4552

In [20]:
# Write your interpretations below:
print("\nInterpretation of significant predictors (p<0.05):")
for var, pval in model.pvalues.items():
    if var != "Intercept" and pval < 0.05:
        coef = model.params[var]
        direction = "positively" if coef > 0 else "negatively"
        print(f"- {var} is {direction} associated with caps_intake (p={pval:.3f}).")
print("Those who are disabled and unemployed had higher CAPS intake scores compared to employed individuals. On the other hand, those with high ranks had lower CAPS scores. Those with higher CTQ scores also had higher CAPS intake scores. These results indicate that PTSD symptom may vary based on factors including employment status, rank, and childhood trauma")


Interpretation of significant predictors (p<0.05):
- C(employment_code)[T.4] is positively associated with caps_intake (p=0.029).
- C(rank_code)[T.2] is negatively associated with caps_intake (p=0.022).
- ctq_total_score is positively associated with caps_intake (p=0.035).
Those who are disabled and unemployed had higher CAPS intake scores compared to employed individuals. On the other hand, those with high ranks had lower CAPS scores. Those with higher CTQ scores also had higher CAPS intake scores. These results indicate that PTSD symptom may vary based on factors including employment status, rank, and childhood trauma
